# 마스크드 모델 구현

<br>

## 구글 BERT의 마스크드 언어 모델(Masked Language Model)

In [ ]:
!pip install transformers

<br>

### 마스크드 언어 모델과 토크나이저
- `transformers` 패키지를 사용하여 모델과 토크나이저를 로드
- BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로, 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 함
  
  - **A라는 이름의 BERT를 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없음**
  
  $→$ BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면에, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 단어와 맵핑되는 정수 정보 자체가 다르기 때문


<br>

#### `TFBertForMaskedLM.from_pretrained('BERT 모델 이름')`
- [MASK]라고 되어있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드
  
  $\rightarrow$ **BERT를 마스크드 언어 모델 형태로 로드합니다.**

<br>

#### `AutoTokenizer.from_pretrained('모델 이름')`
- 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드


In [ ]:
from transformers import TFBertForMaskedLM, AutoTokenizer

In [ ]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

- [CLS] 토큰의 정수 인코딩

In [ ]:
tokenizer.cls_token_id

101

- [SEP] 토큰의 정수 인코딩

In [ ]:
tokenizer.sep_token_id

102

- [MASK] 토큰의 정수 인코딩

In [ ]:
tokenizer.mask_token_id

103

<br>

### BERT의 입력
- 임의의 문장

> ''Soccer is a really fun [MASK]'

- 이를 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측
- 마스크드 언어 모델의 예측 결과를 보기위해서 `bert-large-uncased`의 토크나이저를 사용하여 해당 문장을 정수 인코딩


In [ ]:
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [ ]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[ 101, 4715, 2003, 1037, 2428, 4569,  103, 1012,  102]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

- 토크나이저를 통한 정수 인코딩 결과 (`input_ids`)

In [ ]:
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


- 문장을 구분하는 세그먼트 인코딩 결과 (`token_type_ids`)
  - 현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스
  - 만약 문장이 두 개였다면 두번째 문장이 시작되는 구간부터는 1의 시퀀스가 나옴

In [ ]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


- 실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크 (`attention_mask`)
  - 현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스
  - 만약 뒤에 패딩이 있었다면 패딩이 시작되는 구간부터는 0의 시퀀스가 나옴

In [ ]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


<br>

### [MASK] 토큰 예측

#### `transformers.FillMaskPipeline(model, tokenizer)`
- `FillMaskPipeline`은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리

In [ ]:
from transformers import FillMaskPipeline

In [ ]:
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

- 입력 문장으로부터 [MASK]의 위치에 들어갈 수 있는 상위 5개의 후보 단어들을 출력

In [ ]:
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621111273765564,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.2034205198287964,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208595871925354,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630260601639748,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354860711842775,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [ ]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.25628986954689026,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.1728411465883255,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107723414897919,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248973101377487,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046633958816528,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [ ]:
pip('I went to [MASK] this morning.')

[{'score': 0.3573073446750641,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304396867752075,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.1284506916999817,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.06230578571557999,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.046952586621046066,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]

<br>

## 한국어 BERT의 마스크드 언어 모델(Masked Language Model)
- **`klue/bert-base`는 대표적인 한국어 BERT**
- `klue/bert-base`의 마스크드 언어 모델과 토크나이저를 로드

In [ ]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

<br>

### BERT의 입력
- 임의의 문장

> '축구는 정말 재미있는 [MASK]다'

- 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측
- 문장을 정수 인코딩


In [ ]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [ ]:
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


- 세그먼트 인코딩 결과

In [ ]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


- 어텐션 마스크

In [ ]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


<br>

### [MASK] 토큰 예측

In [ ]:
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [ ]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963505625724792,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.02595764957368374,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033959522843361,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924407720565796,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844232954084873,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [ ]:
pip('어벤져스는 정말 재미있는 [MASK]다.')

[{'score': 0.838241696357727,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.028275659307837486,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.017189301550388336,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.01498968992382288,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.00938261579722166,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [ ]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012574166059494,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.06124085560441017,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.0174866896122694,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.016131814569234848,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360787510871887,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]

<br>

## 구글 BERT의 다음 문장 예측(Next Sentence Prediction)

<br>

### 다음 문장 예측 모델과 토크나이저
- `transformers` 패키지를 사용하여 모델과 토크나이저를 로드
- BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야힘

In [ ]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [ ]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

<br>

### BERT의 입력
- 문맥 상으로 실제로 이어지는 두 개의 문장

In [ ]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

- 두 개의 문장을 정수 인코딩

In [ ]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

- 정수 인코딩 결과 확인 (`input_ids`)
  - **101과 102는 각각 [CLS], [SEP]토큰의 정수 인코딩 (특별 토큰)**

In [ ]:
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


In [ ]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)

[CLS] : 101


In [ ]:
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[SEP] : 102


- 위의 정수 인코딩 결과를 다시 인코딩 $\rightarrow$ 현재 입력의 구성을 확인
- BERT에서 두 개의 문장이 입력으로 들어갈 경우에는 맨 앞에는 [CLS] 토큰이 존재
- 첫번째 문장이 끝나면 [SEP] 토큰, 그리고 두번째 문장이 종료되었을 때 다시 추가적으로 [SEP] 토큰이 추가

In [ ]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


- 문장을 구분하는 세그먼트 인코딩 결과를 확인 (`token_type_ids`)
- 0이 연속적으로 등장하다가 어느 순간부터 1이 연속적으로 등장
  
  $\rightarrow$ [CLS] 토큰의 위치부터 첫번째 문장이 끝나고나서 등장한 [SEP] 토큰까지의 위치에는 0
  
  $→$ 다음 두번째 문장부터는 1이 등장

- `token_type_ids`에서는 0과 1로 두 개의 문장을 구분





In [ ]:
print(encoding['token_type_ids'])

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


<br>

### 다음 문장 예측

- `TFBertForNextSentencePrediction`를 통해서 다음 문장을 예측
- **모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전의 값인 logits을 리턴**
- 해당 값을 소프트맥스 함수를 통과시킨 후에 각 레이블에 대한 확률값을 출력


<br>

#### 이어진 두 개의 문장

In [ ]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [ ]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8381912e-06]], shape=(1, 2), dtype=float32)


- **0번 인덱스에 대한 확률값이 1번 인덱스에 대한 확률값보다 훨씬 높음**

  $\rightarrow$ **실질적으로 모델이 예측한 레이블은 0**
  
  $\rightarrow$ 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록, 더 큰 확률값을 가진 인덱스를 리턴

In [ ]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


- 최종 예측 레이블은 0
  
  $\rightarrow$ **BERT가 다음 문장 예측을 학습했을 당시,
  실질적으로 이어지는 두 개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로 두고서 이진 분류로 학습**

  $→$ **두 입력 문장은 서로 이어진 문장**


<br>

#### 이어지지 않는 두 개의 문장

In [ ]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."

In [ ]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


<br>

## 한국어 BERT의 다음 문장 예측(Next Sentence Prediction)
- 모델과 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

In [ ]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


<br>

###  다음 문장 예측

<br>

#### 이어지는 두 개의 문장

In [ ]:
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."

In [ ]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


<br>

#### 이어지지 않는 두 개의 문장

In [ ]:
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"

In [ ]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


<br>

<hr>

<br>

## 센텐스버트(Sentence BERT, SBERT)
- BERT로부터 문장 임베딩을 얻을 수 있는 센텐스버트(Sentence BERT, SBERT)

<br>

### BERT의 문장 임베딩
- BERT로부터 문장 벡터를 얻는 방법은 여러가지 방법이 존재

- **사전 학습된 BERT로부터 문장 벡터를 얻는 방법은 다음과 같이 세 가지**

1. BERT의 [CLS] 토큰의 출력 벡터를 문장 벡터로 간주한다.
2. BERT의 모든 단어의 출력 벡터에 대해서 평균 풀링을 수행한 벡터를 문장 벡터로 간주한다.
3. BERT의 모든 단어의 출력 벡터에 대해서 맥스 풀링을 수행한 벡터를 문장 벡터로 간주한다.


<br>

#### 1)
- 만약 사전 학습된 BERT에 'I love you'라는 문장이 입력된다고 하였을 때,
  
  **이 문장에 대한 벡터를 얻는 첫번째 방법은 [CLS] 토큰의 출력 벡터를 문장 벡터로 간주하는 것**

<img src='https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC1.PNG'>

- 앞서 BERT로 텍스트 분류 문제에서 [CLS] 토큰의 출력 벡터를, 출력층의 입력으로 사용했던 점에 따라

  **이는 [CLS] 토큰이 입력된 문장에 대한 총체적 표현이라고 간주**
  
  $\rightarrow$ **다시 말해 [CLS] 토큰 자체를 입력 문장의 벡터로 간주할 수 있음**

<br>

#### 2)

<img src='https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC2.PNG'>

- **문장 벡터를 얻는 두번째 방법은 [CLS] 토큰만을 사용하는 것이 아니라 BERT의 모든 출력 벡터들을 평균내는 것**
- 단어 벡터들의 평균을 문장 벡터로 간주할 수 있으며, 이는 BERT에서도 적용
  - BERT의 각 단어에 대한 출력 벡터들에 대해서 평균을 내고 이를 문장 벡터 간주
- 위 그림에서는 출력 벡터들의 평균을 'pooling'이라고 표현
  
  $\rightarrow$ 이를 평균 풀링(mean pooling)으로 표현

<br>

#### 3)
- **세번째 방법은 BERT의 각 단어의 출력 벡터들에 대해서 평균 풀링 대신 맥스 풀링을 진행한 벡터를 얻는 것**


- **이때 평균 풀링을 하느냐와 맥스 풀링을 하느냐에 따라서 해당 문장 벡터가 가지는 의미는 다소 다름**

  - **평균 풀링을 얻은 문장 벡터의 경우에는 모든 단어의 의미를 반영하는 쪽에 가깝다면,**
  
     **맥스 풀링을 얻은 문장 벡터의 경우에는 중요한 단어의 의미를 반영하는 쪽**


<br>

### SBERT(센텐스버트, Sentence-BERT)
- SBERT는 기본적으로 BERT의 문장 임베딩의 성능을 우수하게 개선시킨 모델
- SBERT는 위에서 언급한 BERT의 문장 임베딩을 응용하여 BERT를 파인 튜닝

<br>

#### 1) 문장 쌍 분류 태스크로 파인 튜닝

- SBERT를 학습하는 첫번째 방법은 문장 쌍 분류 태스크
- 대표적으로는 NLI(Natural Language Inferencing) 문제
  - **NLI는 두 개의 문장이 주어지면 수반(entailment) 관계인지, 모순(contradiction) 관계인지, 중립(neutral) 관계인지를 맞추는 문제**
- NLI 데이터의 예시입니다.

<br>

<table>
<thead>
<tr>
<th>문장 A</th>
<th>문장 B</th>
<th>레이블</th>
</tr>
</thead>
<tbody>
<tr>
<td>A lady sits on a bench that is against a shopping mall.</td>
<td>A person sits on the seat.</td>
<td>Entailment</td>
</tr>
<tr>
<td>A lady sits on a bench that is against a shopping mall.</td>
<td>A woman is sitting against a building.</td>
<td>Entailment</td>
</tr>
<tr>
<td>A lady sits on a bench that is against a shopping mall.</td>
<td>Nobody is sitting on the bench.</td>
<td>Contradiction</td>
</tr>
<tr>
<td>Two women are embracing while holding to go packages.</td>
<td>The sisters are hugging goodbye while holding to go packages after just eating lunch.</td>
<td>Neutral</td>
</tr>
</tbody>
</table>

<br>

- SBERT는 NLI 데이터를 학습하기 위해 다음과 같은 구조를 가짐

<img src='https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC3.PNG'>


- 우선 문장 A와 문장 B 각각을 BERT의 입력으로 넣고,

  $\rightarrow$ 앞서 BERT의 문장 임베딩을 얻기위한 방식이라고 언급했던 평균 풀링 또는 맥스 풀링을 통해서 각각에 대한 문장 임베딩 벡터를 얻음 (각각 u와 v)
  
  $\rightarrow$ u벡터와 v벡터의 차이 벡터를 구함 (|u-v|)
  
  $\rightarrow$이 세 가지 벡터를 연결(concatenation)
  
  세미콜론(;)을 연결 기호로 한다면 연결된 벡터의 수식은

$$h = (u; v; |u-v|)$$

- 만약 BERT의 문장 임베딩 벡터의 차원이 $n$이라면 세 개의 벡터를 연결한 벡터 $h$의 차원은 $3n$

<img src='https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC4.PNG'>

- **그리고 이 벡터를 출력층으로 보내 다중 클래스 분류 문제로 전환**
- **다시 말해 분류하고자 하는 클래스의 개수가 $k$라면, 가중치 행렬 $3n \times k$의 크기를 가지는 행렬 $W_y$을 곱한 후에 소프트맥스 함수를 통과시키는 것으로 볼 수 있음**

$$o = softmax(W_{y}h)$$

- 마지막으로 실제값에 해당하는 레이블로부터 오차를 줄이는 방식으로 학습



<br>

#### 2) 문장 쌍 회귀 태스크로 파인 튜닝
- SBERT를 학습하는 두번째 방법은 문장 쌍으로 회귀 문제를 푸는 것
  - 대표적으로 STS(Semantic Textual Similarity) 문제
- **STS란 두 개의 문장으로부터 의미적 유사성을 구하는 문제**

- 다음은 STS 데이터의 예시
  - 여기서 레이블은 두 문장의 유사도로 범위값은 0~5
  
<table>
<thead>
<tr>
<th>문장 A</th>
<th>문장 B</th>
<th>레이블</th>
</tr>
</thead>
<tbody>
<tr>
<td>A plane is taking off.</td>
<td>An air plane is taking off.</td>
<td>5.00</td>
</tr>
<tr>
<td>A man is playing a large flute.</td>
<td>A man is playing a flute.</td>
<td>3.80</td>
</tr>
<tr>
<td>A man is spreading shreded cheese on a pizza.</td>
<td>A man is spreading shredded cheese on an uncoo...</td>
<td>3.80</td>
</tr>
<tr>
<td>Three men are playing chess.</td>
<td>Two men are playing chess.</td>
<td>2.60</td>
</tr>
<tr>
<td>A man is playing the cello.</td>
<td>A man seated is playing the cello.</td>
<td>4.25</td>
</tr>
</tbody>
</table>

[한국어 버전의 STS 데이터셋인 KorSTS 데이터셋](https://github.com/kakaobrain/KorNLUDatasets)

<br>

- SBERT는 STS 데이터를 학습하기 위해 다음과 같은 구조를 가짐

<img src='https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC5.PNG'>

- 문장 A와 문장 B 각각을 BERT의 입력으로 넣고,

  $\rightarrow$ 평균 풀링 또는 맥스 풀링을 통해서 각각에 대한 문장 임베딩 벡터를 얻음 (각각 u와 v)
  
  $\rightarrow$ 이 두 벡터의 코사인 유사도를 구함
  
  $\rightarrow$ 해당 유사도와 레이블 유사도와의 평균 제곱 오차(Mean Squared Error, MSE)를 최소화하는 방식으로 학습
  
  $\rightarrow$ 코사인 유사도의 값의 범위는 -1과 1사이므로 위 데이터와 같이 레이블 스코어의 범위가 0~5점라면, 학습 전 해당 레이블들의 값들을 5로 나누어 값의 범위를 줄인 후 학습

- 선택에 따라
  
  **1) 문장 쌍 분류 태스크로만 파인 튜닝 할 수도 있고,**
  
  **2) 문장 쌍 회귀 태스크로만 파인 튜닝 할 수도 있으며,**
  
  **1)을 학습한 후에 2)를 학습하는 전략을 세울 수도 있음**
